# Pulling US-treasury-yields (1-month rates)

In [1]:
from API_Key_Fred import API_key
import requests
import numpy as np 
import pandas as pd


api_key = API_key
series_id = 'DGS1MO'  # 1-Month Treasury Maturity Rate
print(len(API_key))
# Daterange
start_date = '2020-01-01'
end_date = '2024-28-10'

# request
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&start_date={start_date}&end_date={end_date}&file_type=json'

response = requests.get(url)
data = response.json()

# Print the retrieved data
print(data)

32
{'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'observation_start': '1600-01-01', 'observation_end': '9999-12-31', 'units': 'lin', 'output_type': 1, 'file_type': 'json', 'order_by': 'observation_date', 'sort_order': 'asc', 'count': 6068, 'offset': 0, 'limit': 100000, 'observations': [{'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-07-31', 'value': '3.67'}, {'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-08-01', 'value': '3.65'}, {'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-08-02', 'value': '3.65'}, {'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-08-03', 'value': '3.63'}, {'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-08-06', 'value': '3.62'}, {'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-08-07', 'value': '3.63'}, {'realtime_start': '2024-11-02', 'realtime_end': '2024-11-02', 'date': '2001-0

In [2]:
print(type(data))

<class 'dict'>


In [3]:
observations = data['observations']

# df object
df = pd.DataFrame(observations)

# datetime
df['date'] = pd.to_datetime(df['date'])

df.drop(columns=["realtime_start", "realtime_end"], inplace=True)

# check
print(df)

           date value
0    2001-07-31  3.67
1    2001-08-01  3.65
2    2001-08-02  3.65
3    2001-08-03  3.63
4    2001-08-06  3.62
...         ...   ...
6063 2024-10-25  4.89
6064 2024-10-28  4.88
6065 2024-10-29  4.88
6066 2024-10-30  4.87
6067 2024-10-31  4.76

[6068 rows x 2 columns]


### Filling missing values and transforming from daily to hourly

- Missing values will be filled with linear interpolation
- Hourly rates will be created through mapping the daily rate to 24 hours


In [4]:
print("Rows with NaN values:")
print(df[df['value'].isna()])
df['value'] = df['value'].interpolate(method='linear')

# Verify if there are still any NaNs left (just to check)
print(df['value'].isna().sum())
full_date_range = pd.date_range(start=df['date'].min(), end=df['date'].max())

# Reindex the DataFrame to include all dates
df = df.set_index('date').reindex(full_date_range)

# Interpolate values
df['value'] = df['value'].ffill().bfill().interpolate(method='linear')

print(df['value'].isna().sum())

# Reset the index to make 'date' a column again
df = df.reset_index()
df.rename(columns={'index': 'date'}, inplace=True)

hourly_data = []

# Expand each date to 24 hours
for index, row in df.iterrows():
    for hour in range(24):  # 0 to 23
        hourly_data.append({
            'date': row['date'] + pd.Timedelta(hours=hour),
            'value': row['value']
        })

# Create a new DataFrame
df_hourly = pd.DataFrame(hourly_data)

# Display the new DataFrame
print(df_hourly)

Rows with NaN values:
Empty DataFrame
Columns: [date, value]
Index: []
0
0


C:\Users\sghia\AppData\Local\Temp\ipykernel_9356\3031183581.py:3: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df['value'] = df['value'].interpolate(method='linear')
C:\Users\sghia\AppData\Local\Temp\ipykernel_9356\3031183581.py:13: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df['value'] = df['value'].ffill().bfill().interpolate(method='linear')


                      date value
0      2001-07-31 00:00:00  3.67
1      2001-07-31 01:00:00  3.67
2      2001-07-31 02:00:00  3.67
3      2001-07-31 03:00:00  3.67
4      2001-07-31 04:00:00  3.67
...                    ...   ...
203851 2024-10-31 19:00:00  4.76
203852 2024-10-31 20:00:00  4.76
203853 2024-10-31 21:00:00  4.76
203854 2024-10-31 22:00:00  4.76
203855 2024-10-31 23:00:00  4.76

[203856 rows x 2 columns]


In [5]:
print(df_hourly.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203856 entries, 0 to 203855
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    203856 non-null  datetime64[ns]
 1   value   203856 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 3.1+ MB
None


In [11]:
df_test_hourly = df_hourly.copy()

df_test_hourly["value"] = pd.to_numeric(df_test_hourly["value"], errors="coerce")

df_test_hourly["value"] = df_test_hourly["value"]/100

df_test_hourly["value"] = np.exp(df_test_hourly["value"]/720)-1

print("NaNs after conversion:", df_test_hourly["value"].isna().sum())

df_test_hourly['value'] = df_test_hourly['value'].ffill().bfill().interpolate(method='linear')

print("NaNs after conversion:", df_test_hourly["value"].isna().sum())

NaNs after conversion: 7800
NaNs after conversion: 0


### Creating Table in Database and Filling it

In [12]:
import mysql.connector

connection = mysql.connector.connect(
            host="crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com",
            user="Saam",
            passwd="Stitcheson12!",
            database="Crypto"
        )

cursor = connection.cursor()

create_table = """
CREATE TABLE IF NOT EXISTS US_Bond_Yield (
    id INT AUTO_INCREMENT PRIMARY KEY,
    yield DECIMAL(10, 10) NOT NULL,
    date DATETIME NOT NULL UNIQUE
);
"""

try:
    # Execute the query
    cursor.execute(create_table)
    # Commit the changes
    connection.commit()
    print("Table created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

connection.close()


Table created successfully.


In [13]:
connection = mysql.connector.connect(
            host="crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com",
            user="Saam",
            passwd="Stitcheson12!",
            database="Crypto"
        )

cursor = connection.cursor()

insert_statement = """
    INSERT IGNORE INTO US_Bond_Yield (yield, date)
    VALUES (%s, %s)
"""

data = df_test_hourly[['value', 'date']].values.tolist()
cursor.executemany(insert_statement, data)

# Commit and close connection
connection.commit()
cursor.close()
connection.close()
